<a href="https://colab.research.google.com/github/sainivas-99/License-Plate-Detection/blob/main/LPD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ultralytics -qq
!pip install easyocr -qq
!pip install GPUtil -qq
!pip install pytesseract -qq

In [2]:
import os
import shutil
import glob
import yaml
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import cv2

import ultralytics
from ultralytics import YOLO

import easyocr

import xml.etree.ElementTree as xet
from bs4 import BeautifulSoup

import torch
from GPUtil import showUtilization as gpu_usage
from numba import cuda
import pytesseract

In [3]:
def make_folder(path):
    if not os.path.exists(path):
        os.mkdir(path)

def read_xml(file):
    labels = ['filename', 'width', 'height', 'xmin', 'ymin', 'xmax', 'ymax']
    data = BeautifulSoup(file,"xml")
    d = dict()

    for label in labels:
        txt = data.find(label).text
        if all(c.isdigit() for c in txt):
            d[label] = int(txt)
        else:
            d[label] = txt
    return d

def xml_to_txt(path, height, width):

    with open(path, 'r') as f:
        txt = read_xml(f.read())

    xmin = int(txt['xmin'])
    ymin = int(txt['ymin'])
    xmax = int(txt['xmax'])
    ymax = int(txt['ymax'])

    x_center = (xmin+(xmax-xmin)/2)/width
    y_center = (ymin+(ymax-ymin)/2)/height

    w = (xmax-xmin)/width
    h = (ymax-ymin)/height

    return f'0 {x_center} {y_center} {w} {h}'

In [4]:
total_images = glob.glob('/content/drive/MyDrive/Colab Notebooks/ML/datasets/images/*.png')

In [5]:
np.random.seed(123)
indices = np.arange(len(total_images))
random.shuffle(indices)

train_size = int(0.7 * len(indices))
valid_size = int(0.1 * len(indices))
test_size = len(indices) - train_size - valid_size

train_indices = indices[:train_size]
val_indices = indices[train_size:train_size + valid_size]
test_indices = indices[train_size + valid_size:]


In [6]:
make_folder('/content/drive/MyDrive/Colab Notebooks/ML/datasets/output')

In [7]:
sets = ['train','valid','test']
dirs = ['images','labels']
for set in sets:
    path1 = '/content/drive/MyDrive/Colab Notebooks/ML/datasets/output'+f'/{set}'
    make_folder(path1)
    for dir in dirs:
        path2 = path1+f'/{dir}'
        make_folder(path2)
        print(path2)

/content/drive/MyDrive/Colab Notebooks/ML/datasets/output/train/images
/content/drive/MyDrive/Colab Notebooks/ML/datasets/output/train/labels
/content/drive/MyDrive/Colab Notebooks/ML/datasets/output/valid/images
/content/drive/MyDrive/Colab Notebooks/ML/datasets/output/valid/labels
/content/drive/MyDrive/Colab Notebooks/ML/datasets/output/test/images
/content/drive/MyDrive/Colab Notebooks/ML/datasets/output/test/labels


In [8]:
for i, img_path in enumerate(total_images):
    if i in test_indices:
        dataset = 'test'
    elif i in val_indices:
        dataset = 'valid'
    else:
        dataset = 'train'
    dataset_type_path = '/content/drive/MyDrive/Colab Notebooks/ML/datasets/output'+f'/{dataset}'

    image = cv2.imread(img_path)
    img_height, img_width,_ = image.shape

    annotations_path = img_path.replace('images','annotations').replace('png','xml')
    boxes_info = xml_to_txt(annotations_path, img_height, img_width)

    img_file_name = img_path.split('/')[-1]
    txt_file_name = img_file_name.replace('.png','.txt')
    new_img_path = dataset_type_path+'/images/'+img_file_name
    new_label_path = dataset_type_path+'/labels/'+txt_file_name
    shutil.copy2(img_path, new_img_path)

    txt = open(new_label_path,"w")
    txt.write(boxes_info)
    txt.close()


In [9]:
yaml_details =  {
                        'names' : ['vehicle_plate'],
                        'nc' : 1,
                        'train' : '/content/drive/MyDrive/Colab Notebooks/ML/datasets/output/train',
                        'val' : '/content/drive/MyDrive/Colab Notebooks/ML/datasets/output/valid',
                        'test' : '/content/drive/MyDrive/Colab Notebooks/ML/datasets/output/test'
                }

with open('/content/drive/MyDrive/Colab Notebooks/ML/datasets/output/plate.yaml','w+') as file:
    yaml.safe_dump(yaml_details, file)

with open('/content/drive/MyDrive/Colab Notebooks/ML/datasets/output/plate.yaml','r') as file:
    plate_yaml = yaml.safe_load(file)


In [10]:
license_plate_yolo = YOLO('yolov8s.pt')

In [11]:
if torch.cuda.is_available():
    num_gpus = torch.cuda.device_count()
    print(f"Number of available GPUs: {num_gpus}")

    if num_gpus > 1:
      train_device, test_device = 0,1

    else:
      train_device, test_device = 0,0

    for i in range(num_gpus):
      gpu_properties = torch.cuda.get_device_properties(i)
      print(f"\nGPU {i}: {gpu_properties.name}")
      print(f"  Total Memory: {gpu_properties.total_memory / (1024**3):.2f} GB")
      print(f"  CUDA Version: {gpu_properties.major}.{gpu_properties.minor}")

else:
    print('No GPUs. Using CPU')
    train_device, test_device = 'cpu', 'cpu'



Number of available GPUs: 1

GPU 0: Tesla T4
  Total Memory: 14.75 GB
  CUDA Version: 7.5


In [12]:
license_plate_yolo.train(
    data = '/content/drive/MyDrive/Colab Notebooks/ML/datasets/output/plate.yaml',
    task = 'detect',
    epochs = 50,
    batch = -1,
    val = True,
    seed = 42,
    lr0 = 1e-3,
    lrf = 0.01,
    dropout = 0.4,
    optimizer = 'auto',
    name = 'license plate',
    device = train_device
)

Ultralytics YOLOv8.1.34 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/Colab Notebooks/ML/datasets/output/plate.yaml, epochs=50, time=None, patience=100, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=license plate, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.4, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_cro

train: Scanning /content/drive/MyDrive/Colab Notebooks/ML/datasets/output/train/labels... 303 images, 0 backgrounds, 0 corrupt: 100%|██████████| 303/303 [00:02<00:00, 101.37it/s]


train: New cache created: /content/drive/MyDrive/Colab Notebooks/ML/datasets/output/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /content/drive/MyDrive/Colab Notebooks/ML/datasets/output/valid/labels... 43 images, 0 backgrounds, 0 corrupt: 100%|██████████| 43/43 [00:00<00:00, 65.72it/s]


val: New cache created: /content/drive/MyDrive/Colab Notebooks/ML/datasets/output/valid/labels.cache
Plotting labels to runs/detect/license plate/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.001' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.00059375), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/license plate
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all         43         43      0.998      0.953      0.993      0.645



50 epochs completed in 0.112 hours.
Optimizer stripped from runs/detect/license plate/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/license plate/weights/best.pt, 22.5MB

Validating runs/detect/license plate/weights/best.pt...
Ultralytics YOLOv8.1.34 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.89it/s]


                   all         43         43      0.929          1      0.987      0.643
Speed: 0.2ms preprocess, 4.6ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/detect/license plate


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7841c44803a0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [13]:
shutil.move('/content/runs','/content/drive/MyDrive/Colab Notebooks/ML/datasets/output')

'/content/drive/MyDrive/Colab Notebooks/ML/datasets/output/runs'